This assignment mixes understanding the theory seen in class and reading the code of an implementation of the studied algorithms. You will encounter numbered questions while going through the notebook (this file). You will have to hand in the answers to those questions. To this end use the pdf file that you can also find on Toledo and write down your answer in the corresponding boxes.


## Note on interactive notebooks

Interactive notebooks like this one allow users to interleave code and text in the same place. The notebook is divided into cells, these are either text cells or code cells. Code cells have a little play button next to them (in Googel Collab, other editors have it in a different place). You can run the code in a code cell by simply pressing the play button. The play button becomes visible when hovering with your mouse over a code cell. 



# **Assignment 6 - Hidden Markov Model**

In this assignment we will implement the alpha and beta recursions for hidden Markov models (HMM), which you saw in class and have a look at the Viterbi algorithms, as well. 



Our HMM will model a student's prepardness. However, we are not able to directly observe whether a student is prepared or not. We can only have a look at their grades and whether they attend class or not:
- Knowledgeable on the week's topic [0 = not knowledeable, 1 = partially knowledgeable, 2 = very knowledgeable]
This corresponds to the prepardness of a student. We will abreviate this random variale by **K**.
- Grade in the weekly assignment [0, 9]; abbreviated  **G**.
- Attended the weekly lecture [0 = no, 1 = yes]; describes whether a student attended the class in a specific week or not. Abbreviated by **A**.

The **Hidden Markov Model** has now the following structure:
$K_t$ is the unobserved or hidden random variable. The observed variables are $G_t$ and $A_t$.

---



# **Question 1**


**Draw the graphical representation of the HMM with 3 observations!**



---




Before we start coding up a model we will import the numpy library; a popular scientific computing library in Python.

Press the play button to the left of the cell. It appears when you hover with your mouse above the cell.


In [2]:
import numpy as np

## The Model

In a first step we will write down the initial state, the transition matrix, and the observation model using numpy arrays (matrices).

### Initial State

The initial state describes the initial knowledge of the student before the course starts: $p(K_0=k)$ and $k$ ranges from $0$ to $2$. At time step $0$ we do not have any observations!

In [3]:
prob_K = np.array([
    [0.25],
    [0.50],
    [0.25],
  ]
)

Let us make sure that the probabilities sum up to 1:


In [4]:
np.sum(prob_K)==1

True

### Emission/observation model

Let us now define the observation model for the grades given the knowledge of the student. In short: $p(G_t=g|K_t=k)$, where we have $t \geq 1$, $0\leq g \leq 9$, and $0\leq k \leq 2$.

In [5]:
prob_KG = np.array(
  [
    [0.18, 0.16, 0.15, 0.13, 0.11, 0.09, 0.07, 0.05, 0.04, 0.02], 
    [0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10], 
    [0.02, 0.04, 0.05, 0.07, 0.09, 0.11, 0.13, 0.15, 0.16, 0.18], 
  ]
)

Let us again double check that the probabilities sum up to one.





In [6]:
np.sum(prob_KG, axis=1)==1

array([ True,  True,  True])

Similarly, we define the observation model for the probability of attending class given the student's knowledge: $p(A_t=a|K_t=k)$, with $t>0$, $0\leq a \leq 1$, and $0\leq k \leq 2$.


In [7]:
prob_KA = np.array(
  [
    [0.75, 0.25], # not knowledgeable students attend the lesson with p = 0.75
    [0.50, 0.50], # partially knowledgeable students attend the lesson with p = 0.5
    [0.66, 0.34], # very knowledgeable students attend the lesson with p = 0.66
  ]
)

...and double checking again...

In [8]:
np.sum(prob_KG, axis=1)==1

array([ True,  True,  True])

Because we are going to set up our equation with only a single observation model we will combine both observation model into a single observation model representing the joint observation model (given the knowledge of the student): 

In [11]:
prob_KAG = np.einsum('ka,kg->kag',prob_KA,prob_KG)

And we check again  whether the probabilites sum up to 1. Here we need to check for almost identity as the floating point multplications which were performed when combining both observation models introduced small errors.

In [12]:
np.isclose( np.sum(prob_KAG, axis=(1,2)), 1.0)

array([ True,  True,  True])

---

# **Question 2**

**Which conditional probability does the matrix `prob_KAG` encode?**

---



### Transition model

We also write down the transition model $p(K_t=k_t | K_{t-1}= k_{t-1})$, with $0\leq k_t \leq 2$ and also $0\leq k_{t-1} \leq 2$.

In [13]:
prob_KK = np.array(
  [
    [0.5, 0.3, 0.2],
    [0.2, 0.5, 0.3],
    [0.1, 0.4, 0.5],
  ]
)

Making again sure that the probabilities sum up to 1.

In [14]:
np.sum(prob_KK, axis=1)==1

array([ True,  True,  True])

---

# **Question 3**

**What is the probability of $p(K_t=0| K_{t-1}=1)$?**

---

## Recursions

### The Forward Pass

We now code the forward and backward algorithm that you saw in class using numpy array methods.

In [15]:
def forward(initial, transition, emission, observations):
  if not observations:
    alpha = initial
    return alpha
  else:
    corrector = emission[(slice(None),) + observations[-1]] #Slide 23 corrector
    # dimension: [3]
    corrector = np.expand_dims(corrector, 1) # we adjust the dimension here in order to make the matrix multiplications below work
    # dimensions: [3] -> [3,1]

    alpha_minus1 = forward(initial, transition, emission, observations[:-1]) #this is the recursive part of the forward algorithm: before we can compute the current value we need to forward compute the values from the previous time steps
    #dimensions: [3,1]

    predictor = transition @ alpha_minus1 #the sum on slide 23 is actually a matrix multiplication, we use the @ to make this happen in numpy
    #dimensions: [3,3] times [3,1] -> [3,1]

    alpha = corrector * predictor # and we perform a pointwise vector multiplication, multiplying the corresponding elements
    #dimensions: [3,1] * [3,1] -> [3,1]

    return alpha


### The Backward Pass

The backward pass works quite similarly (slide 24)

In [ ]:
def backward(transition, emission, observations):
  if not observations:
    return np.ones(shape=(transition.shape[0], 1))
  else:
    current_emission = emission[(slice(None),) + observations[0]]
    current_emission = np.expand_dims(current_emission, 1)
    beta_plus1 = backward(transition, emission, observations[1:])
    beta = transition @ (beta_plus1 * current_emission)
  return beta

## Filtering and Smoothing
With the forward and backward pass ready, we can now do some filtering and smoothing. Let's use the following observations:

In [ ]:
obs = [(1,6),(1,7),(0,5)]

### Filtering

We simply express the filtering function in terms of the forward pass. Note how we perform an extra normalization, as we did not compute the a normalization constant when doing the forward pass. The filtering function returns: $ P(K_t | obs) = P(K_t | A_1, G_1,\dots, A_t, G_t)$


In [ ]:
def filtering(initial, transition, emission, observations):
  alpha = forward(initial, transition, emission, observations)
  alpha = alpha / np.sum(alpha)
  return alpha

We can now infer the probability of the current hidden state (K) using filtering:

In [ ]:
# the actual filtering command
filtered_state = filtering(initial=prob_K, transition=prob_KK, emission=prob_KAG, observations=obs)
# printing the obtained results
print(filtered_state)

---


# **Question 4**


**What is the probability $p(K_3=1|obs)?$**


---

### Smoothing

Smoothing works similarly, with an additional argument for specifying the target time step.

In [ ]:
def smoothing(timestep, initial, transition, emission, observations):
  # some checks to make sure that the target time step is within the allowed range
  n_observations = len(observations)
  assert timestep>0
  assert timestep<=n_observations

  past_observations = observations[:timestep]
  future_observations = observations[timestep:]

  alpha = forward(initial, transition, emission, past_observations)
  beta = backward(transition, emission, future_observations)

  forward_backward = alpha * beta
  forward_backward = forward_backward / np.sum(forward_backward)

  return forward_backward

---

# **Question 5**

**What is the probability distribution of $p(K_2=k| obs)$?**

Use the code cell below to obtain the distribution.



---


In [ ]:
# write the actual smoothing command (replace the FIXME with the actual code)
smoothed_state =  #################################################################################### FIXME #########################################################################################
# printing the obtained result
print(smoothed_state)

## The Viterbi Algorithm

We finally consider the problem of computing the **most likely sequence of hidden states *given* the observations**, implementing the *Viterbi algorithm*:

---

# **Question 6**

**In the line marked with a huge FIXME comment, use the right numpy function applied to the mu vector in order to get the state in question. Give the line of code in your solution.**

---

In [ ]:
def viterbi_backward(transition, emission, observations):
  if not observations:
    mu = np.ones(transition.shape[0])
    return [mu]
  else:
    current_emission = emission[(slice(None),) + observations[0]]
    current_emission = np.expand_dims(current_emission, 1)
    # we make again sure here that the dimesions of the arrays match for the matrix multiplications below

    # our recursive call now returns two values, the message mu and the list of most probable states
    mus = viterbi_backward(transition, emission, observations[1:])

    #we use the mu from the next time (t+1) step to compute the one from the current time step
    mu = np.einsum("ij,j,j-> ij", transition, np.squeeze(current_emission), np.squeeze(mus[-1]))
    mu = np.max(mu, axis=1) #################################################################################### This should have been the line to FIXME #########################################################################################

    mus.append(mu)
    return mus

def viterbi_forward(initial, transition, emission, observations, mus):
  if not observations:
    viterbi_state = np.squeeze(initial)*np.squeeze(mus[0])
    viterbi_state = np.argmax(viterbi_state)
    return [viterbi_state]
  else:
    corrector = emission[(slice(None),) + observations[-1]] 
    viterbi_states = viterbi_forward(initial, transition, emission, observations[:-1], mus[:-1])

    viterbi_transition = transition[viterbi_states[-1],:]

    viterbi_state = np.squeeze(viterbi_transition)*np.squeeze(corrector)*np.squeeze(mus[-1])
    viterbi_state = np.argmax(viterbi_state)

    viterbi_states.append(viterbi_state)
    return viterbi_states


def viterbi(initial=prob_K, transition=prob_KK, emission=prob_KAG, observations=obs):
  mus = viterbi_backward(transition, emission, observations)
  mus = list(reversed(mus))

  viterbi_states = viterbi_forward(initial, transition, emission, observations, mus)

  return viterbi_states


We will use the following observations to run the Viterbi algorithm:


In [ ]:
viterbi_obs = [(0,3),(1,9),(1,4),(0, 3),(0,9), (0,8), (0, 7), (1, 5), (1, 4), (1, 6)]

---


# **Question 7**

**What are the most likely hidden states at each time step given the observations `viterbi_obs`?**

---


In [ ]:
# write the actual command for executing viterbi (replace the FIXME with the actual code)
most_likely_hidden_state = viterbi(
    initial=prob_K,
    transition=prob_KK,
    emission=prob_KAG,
    observations=viterbi_obs
) 
print(most_likely_hidden_state)